# AI cập nhật dữ liệu mới nhất cho bản tin 8h30

In [66]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import import_gemini
import gemini_analyze_fuction
import gemini_comment_fuction
import gemini_summary_function
import get_and_crawl_data
import plotly_and_upload
import other_source_data

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(import_gemini)
importlib.reload(gemini_analyze_fuction)
importlib.reload(gemini_comment_fuction)
importlib.reload(gemini_summary_function)
importlib.reload(get_and_crawl_data)
importlib.reload(plotly_and_upload)
importlib.reload(other_source_data)

from import_default import *
from import_database import *
from import_other import *
from import_gemini import *
from gemini_analyze_fuction import *
from gemini_comment_fuction import *
from gemini_summary_function import *
from get_and_crawl_data import *
from plotly_and_upload import *
from other_source_data import *

### Phần dữ liệu lịch sử các ticker

#### Dữ liệu từ Mongo và AlphaVantage

- Lấy dữ liệu từ Mongo

In [67]:
date_series = get_mongo_collection(ref_db, 'date_series')
time_series = get_mongo_collection(ref_db, 'time_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')

if date_series['date'].max() < get_today_date():
    new_date = get_today_date()
    date_series = pd.concat([date_series, pd.DataFrame({'date': [new_date]})]).sort_values(by='date', ascending=False).reset_index(drop=True)

In [68]:
projection = {"_id": 0,"date": 1,"ticker": 1,"open": 1,"high": 1,"low": 1,"close": 1,'volume': 1}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

other_ticker_df = get_mongo_collection(stock_db, 'other_ticker', projection=projection)
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

- Lấy dữ liệu từ AlphaVantage

In [69]:
# Lấy dữ liệu các tỷ giá cần thiết từ Alpha Vantage
fx_pairs = [
    ('USD', 'SEK', 'USD_SEK')
]

data_frames = []
for from_curr, to_curr, col_name in fx_pairs:
    df = get_data_from_av(from_curr, to_curr, col_name)
    if df is not None:
        data_frames.append(df)

av_ticker_df = pd.concat(data_frames, axis=1)
av_ticker_df = av_ticker_df.fillna(method='ffill').dropna()

#### Chuẩn bị các dữ liệu cần thiết

- Tính toán DXY từ các cặp tiền

In [70]:
currency_config = {
    'EUR_USD': {'source': 'other_ticker_df', 'weight': -0.576},
    'USD_JPY': {'source': 'other_ticker_df', 'weight': 0.136},
    'GBP_USD': {'source': 'other_ticker_df', 'weight': -0.119},
    'USD_CAD': {'source': 'other_ticker_df', 'weight': 0.091},
    'USD_CHF': {'source': 'other_ticker_df', 'weight': 0.036},
    'USD_SEK': {'source': 'av_ticker_df', 'weight': 0.042}
}

# Tạo DataFrame cho mỗi cặp tiền tệ
currency_data = {}
for pair, config in currency_config.items():
    if config['source'] == 'other_ticker_df':
        df = other_ticker_df[other_ticker_df['ticker'] == pair][['date', 'close']].set_index('date')
        df.columns = [pair]
    else:
        df = av_ticker_df[['USD_SEK']].copy()
        df.columns = [pair]
    
    currency_data[pair] = df

# Kết hợp tất cả dữ liệu tỷ giá
dxy_calculation_df = pd.concat(currency_data.values(), axis=1, sort=True)
dxy_calculation_df = dxy_calculation_df.sort_index().bfill().ffill()

# Tính toán chỉ số DXY theo công thức chuẩn
dxy_base = 50.14348112
dxy_calculation = dxy_base
for pair, config in currency_config.items():
    dxy_calculation *= (dxy_calculation_df[pair] ** config['weight'])

dxy_calculation_df['ticker'] = 'DXY'
dxy_calculation_df['close'] = dxy_calculation
dxy_calculation_df = dxy_calculation_df[['ticker', 'close']].reset_index()

# usd_vnd_df = av_ticker_df[['USD_VND']].rename(columns={'USD_VND': 'close'})
# usd_vnd_df['ticker'] = 'USD_VND'
# usd_vnd_df = usd_vnd_df.reset_index()

- Ghép tất cả vào ticker_dict

In [71]:
FINAL_DAYS = 20
TICKER_CONFIG = {
    # Chỉ số chứng khoán Việt Nam
    'VNINDEX':   {'df': full_index_df, 'market': 'hose', 'type': 'vn'},
    'VN30':      {'df': full_index_df, 'market': 'hose', 'type': 'vn'},
    'VN30F1M':   {'df': full_index_df, 'market': 'derivatives', 'type': 'vn'},
    
    # Chỉ số chứng khoán quốc tế
    'DJI':       {'df': other_ticker_df, 'market': 'us', 'type': 'international'},
    'FTSE':      {'df': other_ticker_df, 'market': 'eu', 'type': 'international'},
    'SSEC':      {'df': other_ticker_df, 'market': 'asia', 'type': 'international'},
    
    # Khác
    'XAU_USD':   {'df': other_ticker_df, 'market': 'commodity', 'type': 'other'},
    'CLZ':       {'df': other_ticker_df, 'market': 'commodity', 'type': 'other'},
    'DXY':       {'df': dxy_calculation_df, 'market': 'fx', 'type': 'other'},
}
FINAL_COLUMNS = ['date', 'ticker', 'close', '1d_diff', '1d_change', 'cum_change', 'market', 'type']


# 2. XỬ LÝ THEO LUỒNG MỚI
final_df_list = []
for ticker, config in TICKER_CONFIG.items():
    # Lấy và sắp xếp dữ liệu nguồn
    source_df = config['df']
    temp_df = source_df[source_df['ticker'] == ticker].sort_values('date', ascending=False).copy()

    # TÍNH TOÁN TRƯỚC trên toàn bộ chuỗi dữ liệu đã sắp xếp để đảm bảo chính xác
    temp_df['1d_diff'] = temp_df['close'][::-1].diff()[::-1].fillna(0)
    temp_df['1d_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)
    
    # Gán các thông tin phân loại
    temp_df['market'] = config['market']
    temp_df['type'] = config['type']

    # CẮT 1 LẦN DUY NHẤT về 20 ngày sau khi đã tính toán xong
    temp_df = temp_df.head(FINAL_DAYS)

    # Tính toán cuối cùng trên 20 dòng đã cắt
    temp_df['cum_change'] = temp_df['1d_change'][::-1].cumsum()[::-1]

    final_df_list.append(temp_df)

# 3. KẾT QUẢ CUỐI CÙNG
daily_8h30_data_df = pd.concat(final_df_list, ignore_index=True)[FINAL_COLUMNS]


### Phần dữ liệu tiền tệ

#### Lấy khung thời gian biểu đồ và dữ liệu từ FiinproX

In [72]:
# Lấy khung thời gian biểu đồ
chart_range = date_series['date'].iloc[:25]

# Đọc tất cả các file FiinProX từ thư mục Downloads
fiinprox_files = glob.glob(os.path.join(str(Path.home() / "Downloads"), "*FiinProX*.xlsx"))

# Lọc lại các file chứa dữ liệu thị trường mở và có số cột > 6
value_omo_fiinprox_files = []
rate_omo_fiinprox_files = []
rate_nhnn_finprox_files = []
for file in fiinprox_files:
    df = pd.read_excel(file)
    omo_name_check = df.applymap(lambda x: isinstance(x, str) and "Nghiệp vụ thị trường mở" in x).any().any()
    rate_name_check = df.applymap(lambda x: isinstance(x, str) and "Lãi suất thống kê của NHNN" in x).any().any()
    if omo_name_check and len(df.columns) > 6:
        value_omo_fiinprox_files.append(file)
    elif omo_name_check and len(df.columns) == 6:
        rate_omo_fiinprox_files.append(file)
    elif rate_name_check:
        rate_nhnn_finprox_files.append(file)
if len(value_omo_fiinprox_files) == 0:
    raise FileNotFoundError("❌ Chưa tải dữ liệu giá trị OMO từ FiinProX.")
elif len(rate_omo_fiinprox_files) == 0:
    raise FileNotFoundError("❌ Chưa tải dữ liệu lãi suất OMO từ FiinProX.")   
elif len(rate_nhnn_finprox_files) == 0:
    raise FileNotFoundError("❌ Chưa tải dữ liệu lãi suất thống kê của NHNN từ FiinProX.")

#### Dữ liệu cho bảng MM

In [73]:
# Sắp xếp fiinprox_files theo thời gian tạo (tăng dần) và lấy file mới nhất
last_value_omo_fiinprox_file = sorted(value_omo_fiinprox_files, key=os.path.getctime)[-1]
value_omo_df = pd.read_excel(last_value_omo_fiinprox_file, skiprows=4)

#Đọc dữ liệu daily OMO từ file mới nhất
daily_omo_df = value_omo_df.iloc[[1, 2, 5, 6]].reset_index(drop=True)
daily_omo_df = daily_omo_df.transpose().iloc[1:]
daily_omo_df.columns = ['tin_phieu_phat_hanh', 'tin_phieu_dao_han', 'reverse_repo_phat_hanh', 'reverse_repo_dao_han']
daily_omo_df.index = pd.to_datetime(daily_omo_df.index)
daily_omo_df = daily_omo_df[daily_omo_df.index.isin(chart_range)]
daily_omo_df = daily_omo_df.sort_index(ascending=False).fillna(0)
daily_omo_df['omo_bom_tien'] = daily_omo_df['tin_phieu_dao_han'] + daily_omo_df['reverse_repo_phat_hanh']
daily_omo_df['omo_hut_tien'] = daily_omo_df['tin_phieu_phat_hanh'] + daily_omo_df['reverse_repo_dao_han']
daily_omo_df['omo_bom_hut_rong'] = daily_omo_df['omo_bom_tien'] + daily_omo_df['omo_hut_tien']
daily_omo_df = daily_omo_df.iloc[:, [4, 5, 6]]

#Đọc dữ liệu outstanding OMO từ file mới nhất
outstanding_omo_df = value_omo_df.iloc[[3, 7]].reset_index(drop=True)
outstanding_omo_df = outstanding_omo_df.transpose().iloc[1:].fillna(0)
outstanding_omo_df.columns = ['tin_phieu_luu_hanh', 'reverse_repo_luu_hanh']
outstanding_omo_df.index = pd.to_datetime(outstanding_omo_df.index)
outstanding_omo_df['outstanding_omo_value'] = outstanding_omo_df['tin_phieu_luu_hanh'] + outstanding_omo_df['reverse_repo_luu_hanh']
outstanding_omo_df = outstanding_omo_df.sort_index(ascending=False)
outstanding_omo_df = outstanding_omo_df[outstanding_omo_df.index.isin(chart_range)]

#Đọc dữ liệu LSLNH từ file mới nhất
lslnh_df = pd.read_excel(last_value_omo_fiinprox_file, skiprows=4)
lslnh_df = lslnh_df.iloc[[16, 19, 20]].reset_index(drop=True)
lslnh_df = lslnh_df.transpose().iloc[1:]
lslnh_df.columns = ['lslnh_on', 'lslnh_1m', 'lslnh_3m']
lslnh_df.index = pd.to_datetime(lslnh_df.index)
lslnh_df = lslnh_df.sort_index(ascending=False)
lslnh_df = lslnh_df[lslnh_df.index.isin(chart_range)]

#Đọc dữ liệu tỉ giá từ Vietnambiz
for _ in range (5):
    try:
        dhtg_df = fetch_wichart_data(wichart_api_url_dict['tien_te'] + 'dhtg').set_index('date')
        dhtg_df.columns = ['dhtg_trung_tam', 'dhtg_tran', 'dhtg_san', 'dhtg_nhtm_ban_ra', 'dhtg_tu_do_ban_ra']
        dhtg_df = dhtg_df[['dhtg_trung_tam', 'dhtg_tran', 'dhtg_nhtm_ban_ra', 'dhtg_san', 'dhtg_tu_do_ban_ra']]
        dhtg_df.index = pd.to_datetime(dhtg_df.index)
        dhtg_df = dhtg_df[dhtg_df.index.isin(chart_range)]
        break
    except Exception as e:
        raise Exception(f"Lỗi khi lấy dữ liệu từ Wichart: {e}")
    
# Kết hợp tất cả các bảng dữ liệu lãi suất và tiền tệ
daily_8h30_mm_df = pd.concat([lslnh_df, dhtg_df, daily_omo_df, outstanding_omo_df], axis=1, join='outer')
daily_8h30_mm_df = daily_8h30_mm_df.sort_index(ascending=False).bfill().reset_index().rename(columns={'index': 'date'})

#### Dữ liệu cho bảng OMO

In [74]:
# Khởi tạo dictionary lưu file OMO
bill_omo_files = {}
rp_omo_files = {}

# Đọc dữ liệu lãi suất OMO từ các file FiinProX
temp_rate_omo_dict = {}
for file in rate_omo_fiinprox_files:
    df = pd.read_excel(file, skiprows=4).dropna()
    df = df.iloc[:, [0, 3, 4, 5]]
    df.columns = ['date', 'value', 'rate', 'term']
    df['date'] = pd.to_datetime(df['date'])
    df['term'] = df['term'].str.replace(' ngày', '').astype(int)
    temp_rate_omo_dict[file] = df

# Tìm ngày cuối cùng chung của các file OMO
omo_last_date_list = []
for df in temp_rate_omo_dict.values():
    omo_last_date_list.append(df['date'].max())
omo_last_date = min(omo_last_date_list)

# Tính trung bình kỳ hạn cho từng file OMO
omo_mean_term_dict = {}
for file, df in temp_rate_omo_dict.items():
    temp_df = df[df['date'] <= omo_last_date]
    omo_mean_term_dict[file] = temp_df['term'].mean()

# Phân loại file OMO thành bill và repo
bill_omo_files, repo_omo_files = clasify_omo_rate_df(omo_mean_term_dict)

# Lấy file bill và repo mới nhất
last_bill_omo_file = sorted(bill_omo_files, key=os.path.getctime)[-1]
last_repo_omo_file = sorted(repo_omo_files, key=os.path.getctime)[-1]

# Khởi tạo DataFrame lưu dữ liệu đáo hạn và phát hành OMO
issue_omo_df = pd.DataFrame()
for name, file in zip(['bill', 'repo'], [last_bill_omo_file, last_repo_omo_file]):
    df = pd.read_excel(file, skiprows=4).dropna()
    df = df.iloc[:, [0, 3, 4, 5]]
    df.columns = ['date', 'value', 'rate', 'term']
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = df['value'].apply(lambda x: -x if name == 'repo' else x)
    df['term'] = df['term'].str.replace(' ngày', '').astype(int)
    df['term'] = df['term'].apply(lambda x: 7 if x < 10 else x)
    df['type'] = name
    issue_omo_df = pd.concat([issue_omo_df, df], ignore_index=True)

# Chuẩn hóa dữ liệu phát hành OMO
daily_8h30_omo_df = transform_rate_omo_data(issue_omo_df)
daily_8h30_omo_df = daily_8h30_omo_df.reset_index().rename(columns={'index': 'name'})

#### Dữ liệu cho bảng NHTM

In [75]:
last_rate_nhnn_finprox_files = sorted(rate_nhnn_finprox_files, key=os.path.getctime)[-1]
rate_nhnn_df = pd.read_excel(last_rate_nhnn_finprox_files, skiprows=4)
rate_nhnn_df = rate_nhnn_df.iloc[[1, 2, 6, 7]].transpose().iloc[1:]
rate_nhnn_df.columns = ['lshd_san', 'lshd_tran', 'lscv_san', 'lscv_tran']
rate_nhnn_df['lshd_tb'] = (rate_nhnn_df['lshd_san'] + rate_nhnn_df['lshd_tran']) / 2
rate_nhnn_df['lscv_tb'] = (rate_nhnn_df['lscv_san'] + rate_nhnn_df['lscv_tran']) / 2
rate_nhnn_df.index = pd.to_datetime(rate_nhnn_df.index)
rate_nhnn_df = rate_nhnn_df[['lshd_san', 'lshd_tran', 'lshd_tb', 'lscv_san', 'lscv_tran', 'lscv_tb']]

dtnh_df = fetch_wichart_data(wichart_api_url_dict['tien_te'] + 'dtnh').set_index('date')
dtnh_df.index = pd.to_datetime(dtnh_df.index)
dtnh_df.index += pd.Timedelta(days=10)
dtnh_df.index = dtnh_df.index.to_period('M').to_timestamp()
dtnh_df.columns = ['dtnh']

# Fill thêm dữ liệu vào các tháng chưa có
new_index = pd.date_range(rate_nhnn_df.index.min(), date_series['date'].max(), freq='MS').sort_values()
rate_nhnn_df_reindexed = rate_nhnn_df.reindex(new_index)
dtnh_df_reindexed = dtnh_df.reindex(new_index)
daily_8h30_nhtm_df = pd.concat([rate_nhnn_df_reindexed, dtnh_df_reindexed], axis=1).ffill().sort_index(ascending=False)
daily_8h30_nhtm_df = daily_8h30_nhtm_df.reset_index().rename(columns={'index': 'date'})

### Phần dữ liệu cho tin tức

#### Khởi tạo model và danh sách link báo

In [76]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))

# Sắp xếp danh sách model theo thứ tự ổn định
# fast_model_list = select_fast_models(get_gemini_models())
# standard_model_list = select_standard_models(get_gemini_models())
fast_model_list = ['gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']
standard_model_list = ['gemini-2.5-flash', 'gemini-2.0-flash', 'gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

In [77]:
article_url_dict = {
    'VietStock': {
        'https://vietstock.vn/775/the-gioi/kinh-te-nganh.rss': 3,
        'https://vietstock.vn/768/kinh-te/kinh-te-dau-tu.rss': 3,
        'https://vietstock.vn/830/chung-khoan/co-phieu.rss': 3,
        'https://vietstock.vn/761/kinh-te/vi-mo.rss': 3,
        'https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss': 3,
        'https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss': 3,
        'https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss': 3,
    },
    'CafeF': {
        'https://cafef.vn/vi-mo-dau-tu.chn': 5,
        'https://cafef.vn/thi-truong-chung-khoan.chn': 5,
        'https://cafef.vn/tai-chinh-quoc-te.chn': 5,
        'https://cafef.vn/doanh-nghiep.chn': 5,
    },
    'Vietnambiz': {
        'https://vietnambiz.vn/tai-chinh.htm': 5,
        'https://vietnambiz.vn/quoc-te.htm': 5,
        'https://vietnambiz.vn/chung-khoan.htm': 5,
        'https://vietnambiz.vn/doanh-nghiep.htm': 5,
    },
    'VnEconomy': {
        'https://vneconomy.vn/kinh-te-the-gioi.rss': 5,
        'https://vneconomy.vn/tai-chinh.rss': 5,
        'https://vneconomy.vn/chung-khoan.rss': 5,
    }
}

#### Lấy các bài báo từ website

In [78]:
# 1. Tạo dictionary cấu hình để quản lý cả 2 pattern xử lý
source_handlers = {
    'VnEconomy': {
        'type': 'batch',
        'process_batch': get_article_vneconomy 
    },
    'VietStock': {
        'type': 'item_by_item',
        'get_articles': lambda url, num: feedparser.parse(url).entries[:num],
        'get_details': lambda entry: get_article_vietstock(entry['id']),
        'get_published_time': lambda entry: getattr(entry, 'published', '') or ''
    },
    'CafeF': {
        'type': 'item_by_item',
        'get_articles': get_cafef_articles_list,
        'get_details': lambda entry: get_article_cafef(entry['id']),
        'get_published_time': lambda entry: get_cafef_published_time(entry['id'])
    },
    'Vietnambiz': {
        'type': 'item_by_item',
        'get_articles': get_vietnambiz_articles_list,
        'get_details': lambda entry: get_article_vietnambiz(entry['id']),
        'get_published_time': lambda entry: get_vietnambiz_published_time(entry['id'])
    }
}

# 2. Vòng lặp xử lý chính, giờ đây đã trở nên gọn gàng
raw_news_list = []
for source, rss_list in article_url_dict.items():
    for rss_url, num_articles in rss_list.items():
        
        handler = source_handlers.get(source)
        if not handler:
            print(f"Warning: No handler found for source '{source}'. Skipping.")
            continue

        # Phân luồng xử lý dựa trên 'type' đã định nghĩa trong handler
        if handler['type'] == 'batch':
            # Xử lý các nguồn trả về một danh sách hoàn chỉnh
            temp_news_list = handler['process_batch'](rss_url, num_articles)
            raw_news_list.extend(temp_news_list) # Dùng extend hiệu quả hơn là +

        elif handler['type'] == 'item_by_item':
            # Xử lý các nguồn cần lấy chi tiết từng tin
            feed_entries = handler['get_articles'](rss_url, num_articles)
            for entry in feed_entries:
                content, image_url = handler['get_details'](entry)
                published_time = handler['get_published_time'](entry)
                
                raw_news_list.append({
                    'source': source,
                    'title': entry['title'], 
                    'content': content,
                    'image_url': image_url,
                    'article_url': entry['id'],
                    'published_time': published_time,
                })

# 3. Chuyển đổi danh sách tin thành DataFrame
raw_news_df = pd.DataFrame(raw_news_list)
raw_news_df['published_time'] = raw_news_df['published_time'].apply(convert_published_time)

#### Lọc, tóm tắt và chọn tin nổi bật

In [79]:
#Lọc bớt các tin có tiêu đề dài > 20 từ
raw_news_df = raw_news_df[(raw_news_df['title'].str.split().str.len() <= 22) & (raw_news_df['title'].str.split().str.len() > 8)].reset_index(drop=True)

# Lọc và phân loại các tin nổi bật
fithered_news_df = raw_news_df.copy()
filtered_news_index_dict = get_filtered_news_index(standard_model_dict, fithered_news_df, num_articles=10)

# Tạo dictionary ánh xạ từ tên nhóm đến index của các tin nổi bật
news_type_map = {}
for k, idx_list in filtered_news_index_dict.items():
    for idx in idx_list:
        news_type_map[idx] = k

# Thêm cột news_type vào fithered_news_df và lọc ra các dòng có news_type
fithered_news_df['news_type'] = fithered_news_df.index.map(news_type_map)
fithered_news_df = fithered_news_df[fithered_news_df['news_type'].notnull()]
fithered_news_df = fithered_news_df.sort_values(by=['news_type', 'published_time'], ascending=[False, False]).reset_index(drop=True)

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm get_filtered_news_index.


In [80]:
# Tạo danh sách tin hàng tuần với nội dung tóm tắt
daily_8h30_news_df = fithered_news_df.copy()
daily_8h30_news_df['content'] = daily_8h30_news_df['content'].apply(lambda x: summary_daily_article(fast_model_dict, x))

# Thêm các cột cần thiết
daily_8h30_news_df['word_count'] = daily_8h30_news_df['content'].str.split().str.len()
daily_8h30_news_df['impact'] = analyze_news_impact(standard_model_dict, daily_8h30_news_df)

✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-

In [81]:
# Lựa chọn 1 tin tức nổi bật nhất cho mỗi nhóm
daily_8h30_news_df['major_selected'] = identify_major_selected(standard_model_dict, daily_8h30_news_df)

# Lựa chọn 3 tin tức phụ cho mỗi nhóm
top_news_index_list = []
for news_type in daily_8h30_news_df['news_type'].unique():
    temp_news_index_list = get_daily_top_news(standard_model_dict, daily_8h30_news_df, news_type, num_articles = 3)
    top_news_index_list = top_news_index_list + temp_news_index_list
daily_8h30_news_df['sub_selected'] = daily_8h30_news_df.index.isin(top_news_index_list)
daily_8h30_news_df['sub_selected'] = daily_8h30_news_df['sub_selected'].apply(lambda x: 'x' if x else '')

# Gộp các tin được chọn vào cột ai_selected
daily_8h30_news_df['ai_selected'] = daily_8h30_news_df.apply(lambda x: 'o' if x['major_selected'] == 'x' else ('x' if x['sub_selected'] == 'x' else ''), axis=1)
daily_8h30_news_df = daily_8h30_news_df.drop(columns=['major_selected', 'sub_selected'])

# Sắp xếp lại các cột
daily_8h30_news_df['impact'] = pd.Categorical(daily_8h30_news_df['impact'], categories=['TÍCH CỰC', 'TRUNG LẬP', 'TIÊU CỰC'], ordered=True)
daily_8h30_news_df.sort_values(by=['news_type', 'impact'], ascending=[False, True], inplace=True)

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm identify_major_selected.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm identify_major_selected.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm identify_major_selected.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm get_daily_top_news.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm get_daily_top_news.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm get_daily_top_news.


### Lưu dữ liệu vào MSSQL

In [82]:
daily_8h30_time_df = pd.DataFrame([{
    'time': time_series.iloc[0].item().strftime('%H:%M'),
    'date': time_series.iloc[0].item().strftime('%d/%m/%Y')
}])

In [83]:
%%capture
save_to_mssql(cts_engine, daily_8h30_data_df, 'daily_8h30_data')
save_to_mssql(cts_engine, daily_8h30_mm_df, 'daily_8h30_mm')
save_to_mssql(cts_engine, daily_8h30_omo_df, 'daily_8h30_omo')
save_to_mssql(cts_engine, daily_8h30_news_df, 'daily_8h30_news')
save_to_mssql(cts_engine, daily_8h30_time_df, 'daily_8h30_time')
